In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
globalAA = pd.read_csv('queries_results/clustering/A.csv', header = 0, sep=' *, *')
# globalA["Time"]=pd.to_datetime(globalA['MonthYear'], format='%Y%m').dt.strftime('%Y-%m')

/home/micha/anaconda3/envs/geo_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
# droping events without Actor 1 Country Code
nan = globalAA[globalAA.Actor1CountryCode.isna()].iloc[0].name
globalAA = globalAA.drop(globalAA.index[nan])

In [4]:
import geopandas

world = geopandas.read_file(
    geopandas.datasets.get_path('naturalearth_lowres'))

# add lacking country codeds
world.loc[world.name=='France','iso_a3'] = 'FRA'
world.loc[world.name=='Norway','iso_a3'] = 'NOR'
world.loc[world.name=='Kosovo','iso_a3'] = 'FRY'

In [5]:
# droping events without iso_a3 code
globalAA = world.merge(globalAA, left_on='iso_a3', right_on='Actor1CountryCode')

In [6]:
globalA = globalAA[['Events','sumNumMentions','materialConfCoop','verbalConfCoop','avgAvgTone','avgGoldstein','fightCount','expressCount']]

In [7]:
points = globalA
points = points.to_numpy()
points

array([[1.15700000e+03, 5.75900000e+03, 7.69230769e-02, ...,
        1.34433881e+00, 3.20000000e+01, 3.90000000e+01],
       [2.36800000e+03, 1.08160000e+04, 1.01351351e-01, ...,
        1.40950169e+00, 7.10000000e+01, 3.10000000e+01],
       [9.12140000e+04, 4.06209000e+05, 1.16769356e-01, ...,
        9.38683755e-01, 5.13200000e+03, 2.37700000e+03],
       ...,
       [4.22200000e+03, 2.70500000e+04, 1.07531975e-01, ...,
        1.13256750e+00, 1.84000000e+02, 1.09000000e+02],
       [1.46300000e+03, 6.43500000e+03, 1.57894737e-01, ...,
        2.25632262e-01, 1.18000000e+02, 1.60000000e+01],
       [1.86300000e+03, 8.49000000e+03, 1.43853999e-01, ...,
        1.05394525e+00, 1.49000000e+02, 2.00000000e+01]])

## Find clusters using [kMeans()](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

In [8]:
#klasa KMeans z sklearn, konstruktor przyjmuje liczbę klastrów i sposób inicjalizacji centroidów, random_state=1 za
#każdym razem te same punkty startowe
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, init='random', random_state=1)
km = kmeans.fit(points)

In [9]:
#funkcja predict zwaraca etykiety klastru do jakiego został przypisany obiekt
clusters_id = kmeans.predict(points)
clusters_id

array([2, 4, 3, 8, 1, 4, 2, 7, 5, 6, 4, 1, 5, 6, 4, 4, 4, 0, 4, 6, 6, 2,
       7, 4, 9, 6, 6, 5, 4, 2, 4, 1, 4, 6, 2, 5, 4, 2, 3, 1, 1, 6, 6, 2,
       2, 4, 6, 2, 2, 4, 7, 4, 2, 6, 4, 4, 2, 4, 2, 4, 2, 2, 2, 2, 4, 4,
       4, 2, 1, 2, 3, 5, 2, 9, 4, 6, 6, 5, 5, 1, 1, 3, 6, 2, 4, 6, 2, 6,
       6, 6, 9, 2, 7, 5, 2, 6, 7, 9, 2, 4, 2, 0, 9, 6, 5, 6, 3, 9, 6, 1,
       4, 1, 1, 4, 3, 6, 9, 3, 1, 6, 9, 4, 5, 5, 6, 7, 9, 2, 5, 3, 6, 0,
       6, 7, 6, 0, 2, 6, 2, 9, 5, 2, 6, 4, 6, 2, 7, 2, 6, 9, 6, 6, 9, 7,
       6, 2, 1, 4, 1, 1, 2, 4], dtype=int32)

In [10]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = globalAA['Actor1CountryCode']
cluster_map['cluster'] = km.labels_

### Data standardisation with [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [11]:
##jeśli dane są w jakiś sposób skorelowane to znaczy, że występuje redundancja danych, te dane nic nowego nie wnoszą
##są niepotrzebne

## if necessary to scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_points = scaler.fit_transform(points)
scaled_points

array([[-0.26740807, -0.29038989, -0.92774416, ...,  0.5910382 ,
        -0.24779235, -0.2245568 ],
       [-0.24976289, -0.27324304, -0.45554336, ...,  0.67760448,
        -0.24057567, -0.22949127],
       [ 1.04479041,  1.0674226 , -0.15751191, ...,  0.05214176,
         0.69592831,  1.21754422],
       ...,
       [-0.22274871, -0.21819814, -0.33607132, ...,  0.30970853,
        -0.21966578, -0.18138012],
       [-0.26294942, -0.28809777,  0.63744551, ..., -0.89511851,
        -0.23187863, -0.23874342],
       [-0.25712112, -0.28112985,  0.36603675, ...,  0.20526203,
        -0.22614229, -0.23627618]])

## Find clusters using [kMeans()](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) on standardised data

In [12]:
#klasa KMeans z sklearn, konstruktor przyjmuje liczbę klastrów i sposób inicjalizacji centroidów, random_state=1 za
#każdym razem te same punkty startowe
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, init='random', random_state=1)
km = kmeans.fit(scaled_points)

In [13]:
#funkcja predict zwaraca etykiety klastru do jakiego został przypisany obiekt
clusters_id = kmeans.predict(scaled_points)
clusters_id

array([0, 6, 3, 8, 0, 0, 9, 6, 6, 5, 9, 4, 4, 6, 9, 5, 0, 3, 9, 6, 9, 4,
       9, 0, 6, 5, 9, 5, 6, 0, 5, 9, 9, 9, 9, 5, 9, 6, 3, 6, 6, 6, 5, 2,
       9, 6, 9, 0, 0, 4, 5, 4, 6, 6, 6, 5, 2, 6, 0, 4, 9, 9, 6, 0, 6, 5,
       5, 5, 5, 5, 3, 1, 0, 1, 5, 2, 6, 5, 0, 0, 9, 7, 0, 0, 6, 9, 0, 6,
       0, 9, 6, 9, 1, 9, 0, 9, 5, 4, 9, 6, 0, 7, 4, 6, 5, 6, 3, 5, 6, 6,
       6, 6, 6, 0, 3, 6, 6, 3, 6, 6, 6, 0, 0, 9, 0, 6, 6, 9, 6, 3, 6, 3,
       2, 6, 6, 3, 0, 6, 0, 9, 6, 0, 6, 9, 6, 0, 6, 9, 4, 5, 6, 0, 6, 6,
       6, 0, 5, 9, 0, 6, 9, 9], dtype=int32)

In [14]:
cluster_map_std = pd.DataFrame()
cluster_map_std['data_index'] = globalAA['Actor1CountryCode']
cluster_map_std['cluster'] = km.labels_